In [ ]:
!pip install -U transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
!pip uninstall -y datasets
!pip install git+https://github.com/huggingface/datasets.git

Found existing installation: datasets 3.5.0
Uninstalling datasets-3.5.0:
  Successfully uninstalled datasets-3.5.0
  Cloning https://github.com/huggingface/datasets.git to /tmp/pip-req-build-avjvevfv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /tmp/pip-req-build-avjvevfv
  Resolved https://github.com/huggingface/datasets.git to commit ba451d1d9e99904a13413c087a2b01bf64f0e8d6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for datasets: filename=datasets-3.5.1.dev0-py3-none-any.whl size=491554 sha256=5884d9918f1a1c45d39fecf12bf72c8a9d46032f9bda253d5af484375b74e176
  Stored in directory: /tmp/pip-ephem-wheel-cache-dpy8u3qh/wheels/33/07/5d/35c54571205e7fce2b7600c936fd1d5c558c1b7ea98e86e0e1
Successfully built datasets


In [ ]:
import datasets
print(datasets.__version__)
print(datasets.__file__)

3.5.1.dev0
/usr/local/lib/python3.11/dist-packages/datasets/__init__.py


In [ ]:
import transformers
print(transformers.__version__)

4.51.3


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

# Load your dataset
df = pd.read_csv("formal_thai_augmented.csv", encoding="utf-8-sig")

# Prepare the text by adding a prompt
df["text"] = "Summarize this Thai military document carefully. Keep names, dates, locations, procedures accurate: " + df["text"]

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer and model
model_checkpoint = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples["text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(examples["summary"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)


training_args = TrainingArguments(

    output_dir="./mT5-thai-finetuned-V4",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=8,
    weight_decay=0.01,
    fp16=True,
    logging_dir="./logs",
    )


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save the final model
#trainer.save_model("./mT5-thai-finetuned")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/pytho

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

<ipython-input-6-c5863fe743e4>:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: arunyasenadeera (arunyasenadeera-asian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.152100
1000,0.910900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 84, 'num_beams': 4, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1080, training_loss=1.023371661150897, metrics={'train_runtime': 966.9459, 'train_samples_per_second': 1.117, 'train_steps_per_second': 1.117, 'total_flos': 1294969822248960.0, 'train_loss': 1.023371661150897, 'epoch': 8.0})

In [ ]:
# STEP 10: LOGIN TO HUGGINGFACE (one time)
from huggingface_hub import notebook_login
notebook_login()#

In [ ]:
# STEP 11: UPLOAD MODEL TO HUGGINGFACE HUB
from huggingface_hub import HfApi

repo_name = "mT5-thai-summarization-finetuned-v4"  # you can change the name

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

print(f"✅ Successfully uploaded to Hugging Face Hub under: {repo_name}")

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

✅ Successfully uploaded to Hugging Face Hub under: mT5-thai-summarization-finetuned-v4


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "arunyasenadeera/mT5-thai-summarization-finetuned-v4"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Strong summarization function (no post-fix)
def strong_summarize(text, max_input_length=512, max_output_length=700):
    instruction = (
        "โปรดสรุปข้อความต่อไปนี้อย่างกระชับและชัดเจน "
        "รักษาข้อมูลสำคัญให้ครบถ้วน เช่น ชื่อบุคคล ตำแหน่ง หน่วยงาน สถานที่ วันเดือนปี "
        "ห้ามเปลี่ยนแปลงข้อเท็จจริง ห้ามเพิ่มเนื้อหาใหม่ และห้ามตัดข้อมูลสำคัญ "
        "ห้ามละข้อมูลที่เกี่ยวกับการประชุม การเตรียมการ และภารกิจสนับสนุนที่สำคัญ "
        "ใช้ภาษาทางการที่เหมาะสมสำหรับรายงานราชการ."
    )

    input_text = instruction + "\n\n" + text

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_length,
        padding="longest"
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=max_output_length,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example usage:
your_text = """
สปท. (กทด.บก.สปท.) ได้จัดทํา “งานจ้างพัฒนาระบบ Learning Management System และ Online Course” ตามโครงการพัฒนาโครงสร้างพื้นฐานด้านเทคโนโลยีสารสนเทศของ สส.ทหาร ประจําปี งบประมาณ พ.ศ. 2567 ซึ่งเป็นงานดําเนินงานตามสถาปัตยกรรมองค์กร บก.ทท. (RTARF EA) ประจําปี งบประมาณ พ.ศ. 2567 เพื่อใช้ในการสนับสนุนการจัดการศึกษาที่ส่งเสริมการเรียนรู้เชิงรุกให้กับหลักสูตรต่าง ๆ ของ บก.ทท. และ เหล่าทัพ รวมทั้งเพื่อเป็นศูนย์กลางในการเชื่อมต่อข้อมูลด้านการศึกษาและเปิดเผยข้อมูล ด้านการศึกษาตามหลักธรรมาภิบาลข้อมูลภาครัฐ โดยในปัจจุบันได้มีการดําเนินงานจนถึงขั้นตอน การพิจารณาคัดเลือกได้ บริษัท เพอร์เฟคคอมพิวเตอร์โซลูชัน จํากัด เป็นผู้รับจ้างพัฒนาระบบงานฯ ดังกล่าว  เพื่อให้การดําเนินงานเป็นไปด้วยความเรียบร้อย สปท. (กทด.บก.สปท.) จึงขอนัดหมายประชุมเพื่อเก็บรวบรวมข้อมูลและความต้องการใช้งานระบบฯ ในห้วงเดือน มิ.ย.-ก.ค. 67 รายละเอียดตาม QR Code ที่แนบ โดยให้ บริษัท เพอร์เฟคคอมพิวเตอร์โซลูชัน จํากัด ได้เข้าร่วมประชุมฯ หารือและเก็บรวบรวมข้อมูลและความต้องการใช้งานระบบฯ กับหน่วยงานที่จัดหลักสูตรการศึกษา เพื่อให้บริษัทสามารถพัฒนาระบบฯ ได้อย่างมีประสิทธิภาพและสอดคล้องกับความต้องการใช้งานจริง ของหน่วยผู้ใช้งาน ทั้งนี้ มอบหมายให้ พ.ท. ไชยยันต์ วงศาโรจน์ หน.เทคโนโลยีดิจิทัล กทด.บก.สปท. โทร.ทหาร 5031512 โทรศัพท์เคลื่อนที่ 08 5112 3644 เป็นผู้ประสานในรายละเอียด
"""

summary = strong_summarize(your_text)
print("📄 Summary:\n", summary)





tokenizer_config.json:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

📄 Summary:
 กทด.บก.สปท. ขอนัดหมายประชุมเพื่อเก็บรวบรวมข้อมูลและความต้องการใช้งานระบบ Learning Management System และ Online course ในห้วงเดือน มิ.ย.-ก.ค. 67 รายละเอียดตาม QR Code ที่แนบ


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9ce3fbad50fadd4bd97199052dbb86087f8fe8cda96ade67e28b01322f605370
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
from transformers import pipeline
import torch
from evaluate import load

# Load metric
rouge_metric = load("rouge")

# Evaluate mode
model.eval()

# Generate function
def generate_summary_eval(batch):
    inputs = tokenizer(
        batch["text"],
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    ).to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            inputs["input_ids"],
            max_length=512,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    batch["preds"] = preds
    return batch

# Generate predictions
predictions = tokenized_datasets["test"].map(generate_summary_eval, batched=False)

# Prepare predictions and references
pred_texts = [str(p) for p in predictions["preds"]]
ref_texts = [str(r) for r in predictions["summary"]]  # Just normal list of strings

## 📋 Calculate ROUGE
result = rouge_metric.compute(
    predictions=pred_texts,
    references=ref_texts,
    use_stemmer=True
)

# ✅ Correct print format for new evaluate
print(f"ROUGE-1: {result['rouge1']:.4f}")
print(f"ROUGE-2: {result['rouge2']:.4f}")
print(f"ROUGE-L: {result['rougeL']:.4f}")



Map:   0%|          | 0/15 [00:00<?, ? examples/s]

ROUGE-1: 0.4836
ROUGE-2: 0.3827
ROUGE-L: 0.4866
